# LAB Assignment
Please finish the **Exercise** and answer **Questions**.
## Exercise ( Q-Learning with Taxi-v3 🚕) (100 Points)

In this exercise, you should complete the Q-learning algorithm using the Taxi-v3 environment in the gym

In Taxi-v3 🚕, there are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger’s location, picks up the passenger, drives to the passenger’s destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends.
<div align="center"><img src="images/image-20220805133926061.png" alt="image-20220805133926061" style="zoom:80%;" /></div>

## Step 0 Import the packages

In [ ]:
import numpy as np
import gym
import random
import imageio
import tqdm

##  Step 1 Create Taxi-v3 🚕  environment 
Using the API imported from gym

In [ ]:
env = gym.make('Taxi-v3')
env.render()

There are **500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger** (including the case when the passenger is in the taxi), and **4 destination locations.**

In [ ]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

In [ ]:
action_space = env.action_space.n
print("There are ", action_space, " possible actions")

The action space (the set of possible actions the agent can take) is discrete with **6 actions available 🎮**:
- 0: move south
- 1: move north
- 2: move east
- 3: move west
- 4: pickup passenger
- 5: drop off passenger

Reward function 💰:
- -1 per step unless other reward is triggered.
- +20 delivering passenger.
- -10 executing “pickup” and “drop-off” actions illegally.

##  Step 2  Create the Q-table and initialize it

You can use the gym api to fetch the dimension of action space and state space

In [ ]:
action_space = env.action_space.n
state_space = env.observation_space.n

# Please complete this initialization in this line
Q_table = np.zeros((state_space, action_space))

##  Step 3 Configure the hyperparameters

In [ ]:
total_episodes = 100000        # 一共玩多少局游戏
total_test_episodes = 100     # 测试中一共走几步
max_steps = 99                # Max steps per episode 每一局游戏最多走几步

learning_rate = 0.5           # Learning rate
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.05           # Minimum exploration probability 
decay_rate = 0.008            # Exponential decay rate for exploration prob


test_seed = [16,54,165,177,191,191,120,80,149,178,48,38,6,125,174,73,50,172,100,148,146,6,25,40,68,148,49,167,9,97,164,176,61,7,54,55,
 161,131,184,51,170,12,120,113,95,126,51,98,36,135,54,82,45,95,89,59,95,124,9,113,58,85,51,134,121,169,105,21,30,11,50,65,12,43,82,145,152,97,106,55,31,85,38,
 112,102,168,123,97,21,83,158,26,80,63,5,81,32,11,28,148] # Evaluation seed, this ensures that all classmates agents are trained on the same taxi starting position
    

##  Step 4 Q Learning algorithm

Note: The formula of Q table update(Bellman equation)
    ![Bellman equation](https://raw.githubusercontent.com/hanruihua/NoteBook/master/AI-Note/equation.png)

In [ ]:
bar = tqdm.tqdm(total=total_episodes)
sample_rewards = []
for episode in range(total_episodes):
    state= env.reset()
    step=0
    done=False
    sample_reward = 0 
    while True:
        action = np.argmax(Q_table[state, :])
        
        new_state, reward, done, info = env.step(action)
        # Calculate the reward of this episode
        sample_reward += reward
        
        # Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        Q_table[state, action] = Q_table[state, action] + learning_rate * (reward + gamma * np.max(Q_table[new_state, :]) - Q_table[state, action])
        
        # Update the state
        state = new_state
        
        #store the episode reward
        if done == True:
            sample_rewards.append(sample_reward)
            break
    # Reduced exploration probability (due to decreasing uncertainty)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)         
    # print the average reward over 1000 episodes
    if episode%1000 == 0:
        mean_reward = np.mean(sample_rewards)
        sample_rewards = []
        #print(str(episode)+": average reward:" + str(mean_reward))
        bar.set_description(str(episode)+": average reward:" + str(mean_reward))
    bar.update()

##  Step 5 Model evaluation

In [ ]:
fps=5
bar = tqdm.tqdm(total=total_test_episodes)
env.reset()
rewards=[]
images = [] 
for episode in range(total_test_episodes):
    state = env.reset(seed=test_seed[episode])
    step = 0
    done =False
    total_rewards = 0
   
    for step in range(max_steps):
        img = env.render(mode='rgb_array')
        images.append(img)
        action = np.argmax(Q_table[state, :])
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        if done:
            rewards.append(total_rewards)
            break
        state = new_state
     
env.close()
mean_reward = np.mean(rewards)
std_reward = np.std(rewards)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")
imageio.mimsave('taxi-v3.gif', [np.array(img) for i, img in enumerate(images)], fps=fps)


##  Step 6 Visualizing the results

In [ ]:
from IPython.display import Image
Image('taxi-v3.gif')